In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date=dbutils.widgets.get("p_file_date")

In [0]:
#For delta
race_results_df=spark.read.format('delta').load(f"{presentation_folder_path}/race_results")
display(race_results_df)

In [0]:
# race_results_df=spark.read.parquet(f"{presentation_folder_path}/race_results")
# display(race_results_df)

In [0]:
from pyspark.sql.functions import * 
constructor_standing_df=race_results_df.groupBy('team','race_year').agg(sum('points').alias('Points'),count(when(race_results_df.position==1,True)).alias('wins'))
display(constructor_standing_df)

In [0]:
from pyspark.sql import Window

constructor_final_standings_df=constructor_standing_df.withColumn('Rank',rank().over(Window().partitionBy('race_year').orderBy(constructor_standing_df.Points.desc(),constructor_standing_df.wins.desc())))
display(constructor_final_standings_df)

In [0]:
#constructor_final_standings_df.write.mode('overwrite').parquet(f"{presentation_folder_path}/constructor_final_standings")

In [0]:
# Save as SQL
#constructor_final_standings_df.write.mode('overwrite').format('parquet').saveAsTable('f1_presentation.constructor_final_standings')

In [0]:
#for delta 
merge_condition="tgt.race_year=src.race_year AND tgt.team=src.team"
merge_delta_data(constructor_final_standings_df,'f1_presentation','constructor_final_standings',presentation_folder_path,merge_condition,'race_year')

In [0]:
%sql
Select * from f1_presentation.constructor_final_standings

In [0]:
dbutils.notebook.exit("success")